In [1]:
#Import pacakges
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np
from spellchecker import SpellChecker 

In [2]:
#Load data into data frame
df_clean = pd.read_csv('dark_chocolate_bar_ratings.csv')
df_clean.head()

,REF,Company (Manufacturer),Company Location,Review Date,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,2454,5150,U.S.A.,2019,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75
1,2458,5150,U.S.A.,2019,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50
2,2454,5150,U.S.A.,2019,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,797,A. Morin,France,2012,Peru,Peru,63%,"4- B,S,C,L","fruity, melon, roasty",3.75
4,797,A. Morin,France,2012,Bolivia,Bolivia,70%,"4- B,S,C,L","vegetal, nutty",3.50


In [3]:
#Created Functions
#Function to print each item of a list
def print_list(list_name):
    for i in list_name:
        print(i)
        
#Function to extract unique values from a list
def unique_values(test_df):
    values_raw = []
    for row in test_df:
        for value in row:
            values_raw.append(value)
    values_unique = sorted(list(set(values_raw)))
    return values_unique

#Function to make list from data frame
def make_list(test_df):
    values_raw = []
    for row in test_df:
        for value in row:
            values_raw.append(value)
    values = sorted(values_raw)
    return values

# Dataset Cleaning
*** 
## Overall Cleaning
***
1. Drop unnecessary columns.
2. Drop observations with null values.
3. Remove leading and trailing spaces.
4. Standardize column names.

### 1. Drop unnecessary columns.

In [4]:
df_clean.drop(columns=['REF', 'Company (Manufacturer)', 'Company Location', 'Review Date'], inplace=True)
df_clean.head()

,Country of Bean Origin,Specific Bean Origin or Bar Name,Cocoa Percent,Ingredients,Most Memorable Characteristics,Rating
0,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75
1,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50
2,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,Peru,Peru,63%,"4- B,S,C,L","fruity, melon, roasty",3.75
4,Bolivia,Bolivia,70%,"4- B,S,C,L","vegetal, nutty",3.50


### 2. Drop observations with null values.

In [5]:
df_clean = df_clean.dropna(axis=0)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2274 entries, 0 to 2361
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Country of Bean Origin            2274 non-null   object 
 1   Specific Bean Origin or Bar Name  2274 non-null   object 
 2   Cocoa Percent                     2274 non-null   object 
 3   Ingredients                       2274 non-null   object 
 4   Most Memorable Characteristics    2274 non-null   object 
 5   Rating                            2274 non-null   float64
dtypes: float64(1), object(5)
memory usage: 124.4+ KB


### 3. Remove leading and trailing spaces.

In [6]:
#Remove leading and trailling spaces from each column
for col in df_clean[['Country of Bean Origin', 'Specific Bean Origin or Bar Name',
       'Cocoa Percent', 'Ingredients', 'Most Memorable Characteristics']]:
    df_clean[col] = df_clean[col].str.strip()

### 4. Standardize column names.

In [7]:
df_clean.rename(columns={'Country of Bean Origin': 'bean_country', 'Specific Bean Origin or Bar Name': 'bean_features',
                        'Cocoa Percent': 'cocoa_percent', 'Ingredients': 'ingredients',  
                         'Most Memorable Characteristics': 'memorable_characteristics', 'Rating': 'rating'}, inplace=True)
df_clean.head()

,bean_country,bean_features,cocoa_percent,ingredients,memorable_characteristics,rating
0,Madagascar,"Bejofo Estate, batch 1",76%,"3- B,S,C","cocoa, blackberry, full body",3.75
1,Dominican Republic,"Zorzal, batch 1",76%,"3- B,S,C","cocoa, vegetal, savory",3.50
2,Tanzania,"Kokoa Kamili, batch 1",76%,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,Peru,Peru,63%,"4- B,S,C,L","fruity, melon, roasty",3.75
4,Bolivia,Bolivia,70%,"4- B,S,C,L","vegetal, nutty",3.50


##  Cleaning Column "Country of Bean Origin"
***
1. Create a master repository of all the countries in the world. 
2. Compare values to the master repository (i.e. list of all countries in the world).
3. Resolve errors as a result of the comparison to the master repository.

### 1. Create a master repository of all the countries in the world. 

In [8]:
#List of current countries in column
total_bean_country_list = df_clean['bean_country'].tolist()
print_list(total_bean_country_list)

Madagascar
Dominican Republic
Tanzania
Peru
Bolivia
Venezuela
Peru
Ecuador
Peru
Brazil
Papua New Guinea
Peru
Madagascar
Burma
Panama
Colombia
Peru
Venezuela
Cuba
Venezuela
Venezuela
Togo
Peru
Togo
Togo
Sao Tome
Peru
Venezuela
Peru
Peru
Peru
Mexico
Fiji
Fiji
Fiji
Fiji
Vanuatu
Vanuatu
Vanuatu
Vanuatu
Ecuador
Ecuador
Mexico
Madagascar
Brazil
Indonesia
Venezuela
Peru
Madagascar
Trinidad
Vietnam
Mexico
Ecuador
Nicaragua
Tanzania
Vietnam
Bolivia
Peru
Dominican Republic
Bolivia
Venezuela
Peru
Nicaragua
Ecuador
Bolivia
Peru
Dominican Republic
Venezuela
Madagascar
Venezuela
Indonesia
Venezuela
Ecuador
Dominican Republic
Venezuela
Papua New Guinea
Ghana
Ghana
Ecuador
Ecuador
Peru
Peru
Peru
Madagascar
Dominican Republic
Papua New Guinea
Venezuela
Belize
Blend
Venezuela
Grenada
Trinidad
Blend
Ecuador
Jamaica
Peru
Blend
Madagascar
Venezuela
Venezuela
Blend
Brazil
Brazil
Brazil
Brazil
Brazil
Dominican Republic
Bolivia
Madagascar
Venezuela
Ecuador
Mexico
Nicaragua
Ecuador
Belize
Brazil
Peru
Dominican

Madagascar
Venezuela
Ecuador
Venezuela
Ecuador
Ecuador
Ecuador
Ecuador
Ecuador
Dominican Republic
Peru
Dominican Republic
Ivory Coast
Ecuador
Madagascar
Trinidad
Venezuela
Peru
Ecuador
St. Lucia
St. Lucia
Colombia
Honduras
Tanzania
St. Lucia
Sao Tome
Tanzania
Venezuela
St. Lucia
Ecuador
St. Lucia
Madagascar
Vietnam
Ecuador
Trinidad
Ecuador
Madagascar
Ecuador
Venezuela
Venezuela
Dominican Republic
Dominican Republic
Bolivia
Bolivia
Nicaragua
Vietnam
Venezuela
Venezuela
Venezuela
Venezuela
Venezuela
Venezuela
Venezuela
Venezuela
Venezuela
Venezuela
India
Honduras
Vietnam
Peru
Peru
Madagascar
Belize
Dominican Republic
Peru
St.Vincent-Grenadines
Belize
Dominican Republic
Ghana
Venezuela
Ecuador
Venezuela
Vietnam
Ecuador
Madagascar
Jamaica
Peru
Colombia
Trinidad
India
Madagascar
Venezuela
Thailand
Thailand
Thailand
Thailand
Dominican Republic
Dominican Republic
Dominican Republic
Dominican Republic
Dominican Republic
Dominican Republic
Dominican Republic
Dominican Republic
Dominican Republi

In [9]:
#Functiont to extact location
def extract_locations(html):
    r = requests.get(html)
    soup = BeautifulSoup(r.content, "html.parser")
    spans = soup.find_all("ul", {"class": "topic-list"})

    locations_list = []

    for span in spans:
        links = span.find_all('a')
        for link in links:
            locations_list.append(link.text)

    return locations_list

In [10]:
#Extract countries list from select website (The Editors of Encyclopaedia Britannica, 2014)
url = "https://www.britannica.com/topic/list-of-countries-1993160"
countries_list = extract_locations(url)

print_list(countries_list)

Afghanistan
Albania
Algeria
Andorra
Angola
Antigua and Barbuda
Argentina
Armenia
Australia
Austria
Azerbaijan
The Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burundi
Cabo Verde
Cambodia
Cameroon
Canada
Central African Republic
Chad
Chile
China
Colombia
Comoros
Congo, Democratic  Republic of the
Congo, Republic of the
Costa Rica
Côte d’Ivoire
Croatia
Cuba
Cyprus
Czech Republic
Denmark
Djibouti
Dominica
Dominican Republic
East Timor
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Fiji
Finland
France
Gabon
The Gambia
Georgia
Germany
Ghana
Greece
Grenada
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kiribati
Korea, North
Korea, South
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Lesotho
Liberia
Libya
Liechtenstein
Lithuania
Luxembourg
Madagascar
Malawi


The countries_list needs to be cleaned before the comparison. 
A few things that stood out to me:
 * Some countries names had the order altered to where the end of the country showed first (e.g. "Sudan, South")
 * Some countries had double spaces in the name (e.g. Saint Vincent and  the Grenadines)
 * Côte d’Ivoire is commonly known as Ivory Coast. 

To clean this data I decided to:
1. Remove double spaces from the values in the column.
2. Fix order and remove commas from the country name
3. Add an optional name for Côte d’Ivoire and Myanmar.

In [11]:
#Determine which countries are effected by errors.
error_countries = []
for country in countries_list:
    if (',' in country) or ('  ' in country) or (country == 'Côte d’Ivoire') or (country == 'Myanmar'):
        error_countries.append(country)
print_list(error_countries)
len(error_countries)

Congo, Democratic  Republic of the
Congo, Republic of the
Côte d’Ivoire
Korea, North
Korea, South
Micronesia,  Federated States of
Myanmar
Saint Vincent and  the Grenadines
Sudan, South


9

From this, we see that 9 countries need to be cleaned.

In [12]:
#Function to clean countries_list
def audit_countries_list(countries):
    #updated_countries = []

    for index, country in enumerate(countries):
        new_country_space = country.replace("  ", " ")
        if new_country_space != countries[index]:
            #print(new_country_space)
            countries[index] = new_country_space
        #if ',' not in new_country_space and (new_country_space != 'Côte d’Ivoire'):
            #updated_countries.append(new_country_space)

        if ', ' in country:
            element_remove = country.find(', ')
            new_country_comma = new_country_space[element_remove+2:] + ' ' + new_country_space[:element_remove]   
            if new_country_comma != countries[index]:
                #print(new_country)
                countries[index] = new_country_comma
                #updated_countries.append(new_country_comma)

        if country == 'Côte d’Ivoire':
            new_country = country + (' (aka Ivory Coast)')
            if new_country != countries[index]:
                #print(new_country)
                countries[index] = new_country
                #updated_countries.append(new_country)
        if country == "Myanmar":
            new_country = country + (' (aka Burma)')
            if new_country != countries[index]:
                #print(new_country)
                countries[index] = new_country
            
    return countries

In [13]:
#Apply changes to column
countries_list = audit_countries_list(countries_list)

### 2. Compare values to the master repository (i.e. list of all countries in the world).

In [14]:
#Create list of unique countries
unique_bean_countries = sorted(df_clean.bean_country.unique())

len(unique_bean_countries)
print_list(unique_bean_countries)

Australia
Belize
Blend
Bolivia
Brazil
Burma
Cameroon
Colombia
Congo
Costa Rica
Cuba
DR Congo
Dominican Republic
Ecuador
El Salvador
Fiji
Gabon
Ghana
Grenada
Guatemala
Haiti
Honduras
India
Indonesia
Ivory Coast
Jamaica
Liberia
Madagascar
Malaysia
Martinique
Mexico
Nicaragua
Nigeria
Panama
Papua New Guinea
Peru
Philippines
Principe
Puerto Rico
Samoa
Sao Tome
Sao Tome & Principe
Sierra Leone
Solomon Islands
Sri Lanka
St. Lucia
St.Vincent-Grenadines
Sulawesi
Sumatra
Suriname
Taiwan
Tanzania
Thailand
Tobago
Togo
Trinidad
U.S.A.
Uganda
Vanuatu
Venezuela
Vietnam


### 3. Resolve errors as a result of the comparison to the master repository.

In [15]:
#Replace "Tobago" with "Trinidad"
df_clean['bean_country'] = df_clean['bean_country'].replace('Tobago', 'Trinidad')

#Replace "Trinidad" with "Trinidad and Tobago"
df_clean['bean_country'] = df_clean['bean_country'].replace('Trinidad', 'Trinidad and Tobago')

#Replace "Principe" with "Sao Tome"
df_clean['bean_country'] = df_clean['bean_country'].replace('Principe', 'Sao Tome')

#Replace Sao Tome" with "Sao Tome and Principe"
df_clean['bean_country'] = df_clean['bean_country'].replace('Sao Tome', 'Sao Tome and Principe')

# Replace 'Congo' with 'Republic of the Congo'
df_clean['bean_country'] = df_clean['bean_country'].replace('Congo', 'Republic of the Congo')

#Repalce 'DR Republic of the Congo' and 'DR Congo' with 'Democratic Republic of the Congo'
df_clean['bean_country'] = df_clean['bean_country'].replace({'|'.join(['DR Republic of the Congo', 'DR Congo']):'Democratic Republic of the Congo'},regex=True)

#Replace "Sulawesi" and "Sumatra" changed to "Indonesia"
df_clean['bean_country'] = df_clean['bean_country'].replace({'|'.join(['Sulawesi','Sumatra']):'Indonesia'},regex=True)

#Replace remaing countries
dict_country_replace = {'Blend' : 'Multiple Countries', 'Burma' : 'Myanmar (aka Burma)', 'Ivory Coast' : 'Côte d’Ivoire (aka Ivory Coast)',
                       'Martinique' : 'Martinique (French Territory)', 'Puerto Rico' : 'Puerto Rico (United States Territory)',
                       'St. Lucia' : 'Saint Lucia', 'St.Vincent-Grenadines' : 'Saint Vincent and the Grenadines',
                       'U.S.A.' : 'United States', 'Sao Tome & Principe' : 'Sao Tome and Principe'}

df_clean = df_clean.replace({'bean_country' : dict_country_replace}, regex=True)

If the changes are made successfully, then I should only see 'Multiple Countries', 'Puerto Rico (United States Territory)', and Martinique (French Territory) as not in the countries_list repository.

In [16]:
#Confirm bean_country updated
for value in df_clean['bean_country'].unique():
    if value not in countries_list:
        print(value)
        
df_clean.tail()

Multiple Countries
Puerto Rico (United States Territory)
Martinique (French Territory)


,bean_country,bean_features,cocoa_percent,ingredients,memorable_characteristics,rating
2357,Multiple Countries,Raw,80%,"4- B,S*,C,Sa","waxy, cloying, vegetal",2.75
2358,Colombia,"APROCAFA, Acandi",75%,"3- B,S,C","strong nutty, marshmallow",3.75
2359,Belize,Maya Mtn,72%,"3- B,S,C","muted, roasty, accessible",3.50
2360,Republic of the Congo,Mountains of the Moon,70%,"3- B,S,C","fatty, mild nuts, mild fruit",3.25
2361,Multiple Countries,"Dry Aged, 30 yr Anniversary bar",75%,"3- B,S,C","fatty, earthy, cocoa",3.00


##  Cleaning Column "Specific Bean Origin or Bar Name"
***
1. Observe possible values in the column.
2. Determine if the column should be dropped.

### 1. Observe possible values in the column.

In [17]:
#Split values by column
features_splited = df_clean['bean_features'].str.replace(',  ', ',').str.replace(', ', ',').str.split(',').tolist()

#List of total features
total_features_list = make_list(features_splited)

#List of unique features
unique_bean_features = list(set(unique_values(features_splited)))

In [18]:
#Determine possible values for 'Specific Bean Origin or Bar Name'
possible_features_test = df_clean.bean_features

#print_list(possible_features_test)
print('Number of unique values in bean_features:', len(unique_bean_features))
print('Total possible values in bean_features:', len(total_features_list))

#Determine highest occurance rate 
for value in unique_bean_features:
    if total_features_list.count(value) > 50:
        print(value + ': ' + str(total_features_list.count(value)))

Number of unique values in bean_features: 1573
Total possible values in bean_features: 3668
Peru: 55
Ecuador: 62
Madagascar: 70


### 2. Determine if the column should be dropped.

Out of 2274 observations, 1573 unique values, and 3668 total values, the value with the highest occurrence came out to be 70 out of 2274 observations (0.03) with most values between the range of 1 to 5. This leads me to believe that 'Specific Bean Origin or Bar Name' has too much variety in values and not enough of a type of value to effectively use for this analysis.

In [19]:
#Drop column 'bean_features'
df_clean.drop(columns=['bean_features'], inplace=True)
df_clean.head()

,bean_country,cocoa_percent,ingredients,memorable_characteristics,rating
0,Madagascar,76%,"3- B,S,C","cocoa, blackberry, full body",3.75
1,Dominican Republic,76%,"3- B,S,C","cocoa, vegetal, savory",3.50
2,Tanzania,76%,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,Peru,63%,"4- B,S,C,L","fruity, melon, roasty",3.75
4,Bolivia,70%,"4- B,S,C,L","vegetal, nutty",3.50


##  Cleaning Column "Cocoa Percent"
***
1. Change the string to float.

### 1. Change the string to float.

In [20]:
#Change string % into float
df_clean['cocoa_percent'] = df_clean['cocoa_percent'].str.rstrip('%').astype('float') / 100

#Confirm changes made
df_clean.head()

,bean_country,cocoa_percent,ingredients,memorable_characteristics,rating
0,Madagascar,0.76,"3- B,S,C","cocoa, blackberry, full body",3.75
1,Dominican Republic,0.76,"3- B,S,C","cocoa, vegetal, savory",3.50
2,Tanzania,0.76,"3- B,S,C","rich cocoa, fatty, bready",3.25
3,Peru,0.63,"4- B,S,C,L","fruity, melon, roasty",3.75
4,Bolivia,0.70,"4- B,S,C,L","vegetal, nutty",3.50


##  Cleaning Column "Ingredients"
***
1. Remove commas from values.
2. Split ingredient amount and ingredients combination into two columns.

### 1. Remove commas from values.

In [21]:
#Remove spaces and commas from 'ingredients'
df_clean.ingredients = df_clean['ingredients'].str.replace(',', '').str.replace(' ', '')

#Confirm changes occured
df_clean.head()

,bean_country,cocoa_percent,ingredients,memorable_characteristics,rating
0,Madagascar,0.76,3-BSC,"cocoa, blackberry, full body",3.75
1,Dominican Republic,0.76,3-BSC,"cocoa, vegetal, savory",3.50
2,Tanzania,0.76,3-BSC,"rich cocoa, fatty, bready",3.25
3,Peru,0.63,4-BSCL,"fruity, melon, roasty",3.75
4,Bolivia,0.70,4-BSCL,"vegetal, nutty",3.50


### 2. Split ingredient amount and ingredient combination into two columns.

In [22]:
#Split 'ingredients' into two column 'total_ingredients' and 'ingredients'
df_clean[['total_ingredients','ingredients']] = df_clean.ingredients.str.split('-',expand=True)

#Confirm changes occured
df_clean.head()

,bean_country,cocoa_percent,ingredients,memorable_characteristics,rating,total_ingredients
0,Madagascar,0.76,BSC,"cocoa, blackberry, full body",3.75,3
1,Dominican Republic,0.76,BSC,"cocoa, vegetal, savory",3.50,3
2,Tanzania,0.76,BSC,"rich cocoa, fatty, bready",3.25,3
3,Peru,0.63,BSCL,"fruity, melon, roasty",3.75,4
4,Bolivia,0.70,BSCL,"vegetal, nutty",3.50,4


In [23]:
#Rename 'ingredients' to 'ingredient_combination'
df_clean.rename(columns={'ingredients': 'ingredient_combination'}, inplace=True)

#Convert 'total_ingredients' to integer
df_clean["total_ingredients"] = pd.to_numeric(df_clean["total_ingredients"])

#Confirm changes occured
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2274 entries, 0 to 2361
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   bean_country               2274 non-null   object 
 1   cocoa_percent              2274 non-null   float64
 2   ingredient_combination     2274 non-null   object 
 3   memorable_characteristics  2274 non-null   object 
 4   rating                     2274 non-null   float64
 5   total_ingredients          2274 non-null   int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 124.4+ KB


##  Cleaning Column "Most Memorable Characteristics"
*** 
1. Clean and standardize values in the dataset (all lowercase, no spaces between characteristics, etc).
2. Create seperate CSV for all possible memorable characteristics.

### 1. Clean and standardize values in the dataset (all lowercase, no spaces between characteristics, etc).

Since each characteristic for observation in a string with one to four possible memorable characteristics, I first glanced over all the possible unique rows to see how a characteristic was separated for each observation. Once I go over the list and note what type of separators are used, then I will separate them into substrings into unique characteristics and unique words.

In [24]:
#Raw characteristic list
pattern = '|'.join([' and ', 'with', 'w/', '&', '/', ' , '])
char_list_raw = df_clean['memorable_characteristics'].str.replace(pattern, ',').str.replace(', ', ',').str.split(',').to_list()

# Characteristics list with duplicates
full_char_list = make_list(char_list_raw)

# Unique characteristics list
unique_char_raw = unique_values(char_list_raw)
unique_char_raw.remove('')

#Create List of unique words for spelling
pattern = '|'.join([' and ', 'with', 'w/', '&', '/',  ' , ', ' '])
word_list = df_clean['memorable_characteristics'].str.replace(pattern, ',').str.split(',').to_list()

unique_words = unique_values(word_list)
unique_words.remove('')

print_list(unique_words)

"Andes"
Burnt
Cadbury
Easter
Fruity
Roasty
XL
accesible
accessible
acid
acidic
acidity
add
aftertaste
alcohol
alkalyzed
alluring
almond
almost
alocohol
aluminum
ambiguous
anise
appealing
apple
aroma
artificial
ashey
assertive
astringcy
astringency
astringent
astringnet
atypical
baked
balanced
banana
bar
base
basic
batter
bean
berries
berry
bitter
bitterness
black
blackberries
blackberry
blackpepper
bland
blueberries
blueberry
bodied
body
bold
bother
bourbon
brandy
bread
bready
brief
bright
brown
brownie
brownine
burlap
burn
burning
burnt
but
butt
butter
butterscotch
buttery
by
cacao
candied
candy
candy-like
caramel
cardamom
cardamon
cardboard
carrots
cashew
chalky
charcoal
cheese
cheesey
cheesy
chemical
cherries
cherry
chewy
chlorine
choco
chocolate
chocolatey
chocolaty
chunky
cigar
cigarette
cinamon
cinnamon
citrus
classic
clingy
clove
cloying
coarse
cocoa
coconut
cocounut
coffee
color
complex
controlled
cooa
cooked
cooling
cough
covered
cranberries
cranberry
cream
creamy
crisp
crumbl

In [25]:
len(unique_char_raw)

848

In [26]:
#Print frequency each value has
for value in unique_char_raw:
    print(value + ': ' + str(full_char_list.count(value)))

"Andes" mint: 1
Cadbury egg: 1
Easter candy: 1
Fruity: 1
Roasty: 1
XL nibs: 1
accesible: 1
accessible: 3
acidic: 36
alcohol: 2
alkalyzed notes: 1
alluring aroma: 1
almond: 2
almond butter: 1
almost burnt: 1
alocohol: 1
aluminum: 1
ambiguous: 1
anise: 5
appealing smoke: 1
apple: 2
artificial: 2
ashey: 12
assertive: 1
astringent: 34
atypical: 2
baked: 1
baked bread: 1
balanced: 16
banana: 44
banana cream: 1
banana nut: 1
banana raspberry: 1
base cocoa: 4
base cocoa : 1
basic: 6
basic chocolate: 1
basic cocoa: 22
basic cocoa base: 1
berries: 3
berry: 19
berry notes: 1
berry notes : 1
bitter: 56
bitter aftertaste: 1
bitter coffee flavor: 1
bitter dominate: 1
bitter ending: 1
bitter then nutty rustic: 1
black current: 1
black licorice: 10
black pepper: 25
black tea: 3
blackberries: 1
blackberry: 10
blackpepper: 1
bland: 18
blueberries: 1
blueberry: 7
bold: 19
bold cherry: 1
bold citrus: 1
bold olive: 1
bourbon: 11
brandy: 1
bread: 4
bread notes: 1
bready: 5
brief fruit note: 1
bright: 1
bri

pungent grape: 1
pungent off flavor: 1
pungent raisin: 1
pure: 3
raisin: 10
raisins: 15
raisiny: 2
raisns: 1
rasin: 1
raspberries: 1
raspberry: 13
raw: 5
red berry: 27
red fruit: 16
red wine: 1
refined: 1
rich: 70
rich base cocoa: 1
rich brownie: 4
rich burnt caramel: 1
rich choco: 4
rich chocolate: 2
rich cocoa: 53
rich cocoa dominates: 1
rich cooa: 1
rich fruit: 1
rich hot cocoa: 1
rich malt: 1
rich mocha: 1
ripe grapes: 1
ripe orange: 1
roast dominates: 1
roaste brownie: 1
roasted: 2
roasted banana: 1
roasted cacao: 1
roasted caramel: 1
roasted cherry: 1
roasted cocoa: 1
roasted nuts: 7
roasted pear: 1
roasted strawberry: 1
roasted vegetal: 1
roasty: 205
roasty cocoa: 1
roasty nibs: 1
robust: 4
robust dark tropical fruit: 1
rough: 1
rounded: 1
rubber: 21
rubber off: 1
rubbery: 24
rum: 14
rustic: 13
salt: 6
salt noticeable: 1
salty: 5
sandpaper: 1
sandy: 158
sandy texture: 1
savory: 13
savory lemon: 1
sharp: 1
short: 6
short length: 2
silky: 2
silky smooth: 1
simple: 13
simple cocoa:

872 unique characteristics are a lot of possibilities. This column does not focus solely on the flavors and/or textures of the chocolate bar. Rather, this column lists the most memorable characteristics whether it be the flavor, texture, or even how the flavors work with each other. I also noticed that since it is not solely focused on flavor, it is important to keep a distinction between 'cocoa' and 'dark cocoa' since they are two different flavors. The same goes for 'nutty' and 'very nutty', these are the characteristics that stood out, and the adjectives used to describe the characteristic should be preserved. 

However, there is a need for standardizing the values. For example, there are misspellings (e.g. ‘accesible’) and variations of the same characteristics (e.g. "cream' and 'creamy'). Values like these need to be updated to correct spelling and simplify the variations to describe the same characteristic.

This will be accomplished in two steps.

*1a. Clean punctuation and spacing within the string of each observation*

*1.b Fix spelling errors and inconsistencies in word choice*

#### 1a. Clean punctuation and spacing within the string of each observation.

In [27]:
#Create and print list of unique rows
unique_row = df_clean['memorable_characteristics'].unique().tolist()
print_list(unique_row)

cocoa, blackberry, full body
cocoa, vegetal, savory
rich cocoa, fatty, bready
fruity, melon, roasty
vegetal, nutty
oily, nut, caramel, raspberry
sweet, cocoa, tangerine
sandy, nutty, cocoa, fig
cocoa,sour,intense tangerine
mild tobacco
mild fruit, strong smoke
green, nutty, cocoa
sticky, red fruit, sour
sticky, smokey, grass
brief fruit note, earthy, nutty
burnt rubber,alkalyzed notes
delicate, hazelnut, brownie
astringent, nutty, chocolatey
sliglty dry, papaya
nutty, mild choco, roasty
harsh, leather, earthy
mild profile, chocolaty, spice
grainy texture, cocoa, sweet
roasty, acidic, nutty
burnt wood, earthy, choco
sweet, chocolatey, vegetal
sweet, dairy, spice, cocoa
alluring aroma, powdery,nutty
yellow fruit
intense, nutty, cherry, cocoa
hot cocoa, rich, molasses
herbal, molasses, hammy
orange, floral, lemon
cocoa and powerful tang
spicy and candy-like
sweet, sublte strawberry
sandy, woody, spicy, sweet
basic, sweet, cocoa, woody
sweet, cocoa, rubbery
unrefined, sweet, metallic
roast

chalky, vanilla, caramel
fatty, powdery, nut, sl. Burnt
vanilla, fatty, basic cocoa
texture issues, brownie
one dimensional, brownie
high roast, fruit
high roast, intense, fatty
dark coffee, cocoa
cinamon, nutmeg, coffee
nutty, simple, sweet
waxy, tobacco, smokey
mild mint, basic cocoa
sandy, cloying, rich cocoa
fatty, rich cocoa, roasty
smokey,chalky,berry,caramel
mild bitter, fatty, grassy
dirty, floral, unrefined
metallic, dirt/ earthy
vanilla, chemical, rubber
smokey, roasty, acidic, rich
red berry, lingering coffee
sticky, woodsy, rum
intense fruit, bitter ending
grassy, tea, late sour
earthy, tart, sour
sandy, flat, hint of berry
sticky, very roasty, nutty
dry, mint, medicinal
butter, floral, rum
oily, black pepper, roasty
mild fruit and spice, roasty
sticky, rich, cherry
tropical berry, intense
well defined, creamy, cherry
blackberry, raisns
sticky, roasty nibs, grapes
cocoa, tea, malt
creamy, sticky, dried fruit
sticky, dried fruit, toast, cocoa
burnt honey, earthy
complex, swe

creamy, smoke, raspberry
raisins, melon
sandy, earthy, fatty
dairy, cocoa, fatty, apple
sandy, grits, nutty, vanilla
leather, salt
sticky, cardboard,why bother
chemical, salt, wtf
strawberry, cream, spice
molassses, palm, spicy
spicy, rich, mint
nutty, grassy, woody
woody, lemon, mild bitter
black pepper, dairy
rum, herbal, rich
banana, melon, fatty
grape, cocoa
candy spices, sweet
complex, true to origin
fatty, simple
wine, brownie batter
unrefined, burnt nuts, vanilla
sandy, nutmeg, astringent
dry, sandy, empty, sweet
dry, sandy, dirt, empty
grape, thick, chewy
cocoa, mild pear, sticky
cocounut, late sour
intense, sour, hammy
dry, coffee notes from roaster
sandy, harsh, floral
gritty, sweet, earty
slight grainy, roasty, spicy
tangy, roasty, ham
few grits, intense smoke
complex, fruit, nut, rich
sour fig, cocoa, creamy, fatty
coarse, smokey, citrus
sticky, blackberry, acidic
powdery, intense, floral
rough, sweet, dairy, vanilla
basic cocoa
strawberry, blueberry
earthy
dirty, rubbery, 

dried fruit, tart
earthy, mild tannic, woody
strawberry, green mint
super temper,astringent, burn
creamy, raisin, lemon
creamy, complex, peanut
cranberries, cocoa
intense, rich burnt caramel
sandy, earthy, tangy wine
smoke,tobacco,black pepper
mild smoke, fruit, mild burnt
uneven, grassy, roasty
pronounced berry,mild smoke
delicate, nutty, cocoa, dairy
roasted nuts, burnt brownie
floral, berry, clove
smoke, burnt caramel, citrus
creamy, brownie mix, roasty
nutty, chocolate ice cream
creamy, smokey, some fruit
grassy, dairy, mild fatty
creamy, nutty, roasty
creamy, medicinal, mild fruit
creamy, sour orange
creamy, smokey, rich cocoa
basic chocolate, accessible
robust, nutty, brownie
spicy, woodsy
tart, mild sour
balanced, nuts, strawberry
roasted nuts, dried fruit
creamy, honey, blackberry
evolves black pepper to citrus
banana, nutty
creamy, sweet, deep choco
robust dark tropical fruit
creamy, diluted, mild choco
nutty, banana, harsh ending
creamy, honey, marshmallow
creamy, roasty, woo

Each memorable characteristic for observation has the following errors the need to be resolved to standardize the format of the strings of this column.
* ', ', 'and', 'with', '/', '&', 'w/'

In [28]:
#Remove uncessary words and characters in strings
pattern = '|'.join([' and ', 'with', 'w/', '&', '/', ', ', ' , '])
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].str.replace(pattern, ',')
#df_clean.head()

#Make all lowercase
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].str.lower()
df_clean.head()

,bean_country,cocoa_percent,ingredient_combination,memorable_characteristics,rating,total_ingredients
0,Madagascar,0.76,BSC,"cocoa,blackberry,full body",3.75,3
1,Dominican Republic,0.76,BSC,"cocoa,vegetal,savory",3.50,3
2,Tanzania,0.76,BSC,"rich cocoa,fatty,bready",3.25,3
3,Peru,0.63,BSCL,"fruity,melon,roasty",3.75,4
4,Bolivia,0.70,BSCL,"vegetal,nutty",3.50,4


#### 1.b Fix spelling errors and inconsistencies in word choice.

In [29]:
#Determine misspelled words
spell_check_df = unique_words
spell = SpellChecker()
misspelled = spell.unknown(spell_check_df)

#Spell check unique_words and compare context to char_list_raw
for word in sorted(misspelled):
    print(word +':', list(spell.candidates(word))[:5])

"andes": ['andes', 'landes']
accesible: ['accessible']
alkalyzed: ['analyzed']
alocohol: ['alcohol']
ashey: ['ashley', 'askey', 'ashby', 'ashy', 'ashe']
astringcy: ['stringy']
astringency: ['stringency']
astringnet: ['astringent']
blackpepper: ['blackpepper']
brownine: ['browning', 'brownie']
burlap: ['burglar', 'burmah', 'burial', 'burra', 'bursar']
cacao: ['macao']
candy-like: ['lady-like']
cardamon: ['cardamom']
cheesey: ['cheesy', 'cheesed', 'cheeses', 'cheese']
choco: ['chico', 'coco', 'choc', 'choo', 'chocs']
chocolatey: ['chocolate', 'chocolates']
chocolaty: ['chocolate']
cinamon: ['cinnamon']
cocounut: ['coconut']
cooa: ['coda', 'coo', 'cora', 'cosa', 'coca']
descript: ['rescript']
distinquished: ['distinguished']
domintates: ['dominates']
dutched?: ['ditched']
earty: ['tarty', 'hearty', 'arty', 'earthy', 'carty']
fertilzer: ['fertilizer']
flavorful: ['flavourful']
flavors: ['flavours', 'flavor']
floral-spice: ['floral-spice']
fudge-like: ['fudge-like']
fudgey: ['fudge', 'fudge

In [30]:
#Function to check char in unique row
def check_char(string):
    for char in unique_char_raw:
        if string in char:
            print(char)

#Function to check char in data frame
def char_in_df(char):
    for value in df_clean['memorable_characteristics']:
        if char in value:
            print(value)

In [31]:
#check_char('ligt')
# Replace 'ligt' with 'light' ('sligt' : 'slight')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('ligt', 'light', regex=True)

# Replace 'sliglty', 'slightly', and 'slighthly' with 'slight'
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['sliglty', 'slightly', 'slighthly']):'slight'},regex=True)

# Replace 'sl.' with 'light'
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('sl. ', 'slight ', regex=True)

#check_char('intens')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['intensely', 'intensity']):'intense'},regex=True)
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('intens', 'intense', regex=True)
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('intensee', 'intense', regex=True)


# Replace 'astringnet', 'astringcy', and 'astringency' with 'slight'
#check_char('astrin')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['astringnet', 'astringcy', 'astringency']):'astringent'},regex=True)

#Fix Chocolate Errors
#check_char('choco')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['chocolatey', 'chocolaty']):'chocolate'},regex=True)
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('choco', 'chocolate', regex=True)
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('chocolatelate', 'chocolate', regex=True)

# Replace 'non descript' and 'non-descript' with 'nondescript'
#check_char('descript')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['non descript', 'non-descript']):'nondescript'},regex=True)

#check_char('domin')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['domintates', 'dominated', 'dominates', 'dominating']):'dominate'},regex=True)
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('dominate', 'dominant', regex=True)

#check_char('ear')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['earty', 'earthen', 'earthy']):'earth'},regex=True)

#check_char('fudg')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['fudge-like', 'fudgey']):'fudge'},regex=True)

#check_char('ham')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['ham-like', 'hammy']):'ham'},regex=True)

#check_char('her')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['hebs', 'herbs', 'herbal']):'herb'},regex=True)

#check_char('roast')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['roaster', 'roasty']):'roast'},regex=True)
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('roaste ', 'roasted ', regex=True)
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('roast ', 'roasted ', regex=True)

#check_char('rai')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['rasin', 'raisns', 'raisiny', 'raisins']):'rasin'},regex=True)

#check_char('mild')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['mildly', 'mildy']):'mild'},regex=True)

#check_char('smo')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['smomkey', 'smokey']):'smoke'},regex=True)

#check_char('spic')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['spicey', 'spiciness', 'spices']):'spice'},regex=True)

#check_char('like')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['candy like', 'candy-like']):'like candy'},regex=True)
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('brownie like', 'like brownie', regex=True)
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('melon like', 'like melon', regex=True)
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('tea-like', 'like tea', regex=True)

#check_char('grou')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('coffee grounds', 'coffee ground', regex=True)
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace('coffee ground', 'coffee grounds', regex=True)

#check_char('vanill')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].str.replace(r"\s*\([^()]*\)","").str.strip()

#check_char('veg')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['veggie', 'vegetable']):'vegetal'},regex=True)


#check_char('dis')
#check_char('over')
#check_char('mouthfeel')
#check_char('wo')

#Fix reamining spelling issues in data frame
dict_spell_replace = {'accesible':'accessible', 'alkalyzed':'alkalized', 'alocohol':'alcohol', 'ashey':'ashy', 
                      'blackpepper':'black pepper', 'brownine':'brownie', 'cardamon':'cardamom', 'cheesey':'cheesy', 
                      'cinamon':'cinnamon', 'cooa':'cocoa', 'fertilzer':'fertilizer', 'klingy':'clingy', 'lavendar':'lavender', 
                      'metallic':'metal', 'malitol':'maltitol', 'molassses':'molasses', 'moldy':'mold', 'pastey':'pasty', 
                      'peanutbutter':'peanut butter', 'pomegrant':'pomegranate', 'prononced':'pronounced', 'smoth':'smooth', 
                      'spciy':'spicy', 'sticy':'sticky', 'sublte':'subtle', 'winey':'wine', 'woody':'wood', 'nibby':'nibs',
                      'over ':'overly ', 'waxy mouthfeel':'waxy', 'distinquished':'distinct', 'rasin':'raisin' }

df_clean = df_clean.replace({'memorable_characteristics' : dict_spell_replace}, regex=True)


#check_char('nut')
df_clean['memorable_characteristics'] = df_clean['memorable_characteristics'].replace({'|'.join(['nutty', 'nuts']):'nut'},regex=True)

#Clean inconsistent in spelling or synatx in values in data frame
dict_stuc_replace = {'med roast':'medium roast', 'rries':'rry', 'plums':'plum', 'slow develop':'slow to develop', 
                     'fruity':'fruit','bitterness':'bitter', 'body':'bodied', 'bready':'bread',
                    'mushroom, mild burn':'mushroom, mild burning', 'buttery':'butter', 'cheese':'cheesy', 'cocounut': 'coconut', 'ending':'end',
                    'dirty':'dirt', 'fat ':'fatty ', 'fat,':'fatty,', 'grain,': 'grainy,', 'grapes':'grape', 'grassy':'grass',
                    'grit,':'gritty,', 'hints':'hint', 'hit':'hint', 'leathery':'leather', 'minty':'mint', 'notes':'note',
                    'oddly':'odd', 'oranges':'orange', 'overyly': 'overly', 'prunes':'prune', 'perfectly':'perfect', 
                     'rubbery':'rubber','salty': 'salt', 'soapy':'soap', 'sourness':'sour', 'sugary':'sugar', 'tangy':'tang',
                     'watering':'watery', 'burn butterscotch':'burnt butterscotch', 'burning':'burn'}

df_clean = df_clean.replace({'memorable_characteristics' : dict_stuc_replace}, regex=True)

#Confirm changes occured
df_clean.head()

,bean_country,cocoa_percent,ingredient_combination,memorable_characteristics,rating,total_ingredients
0,Madagascar,0.76,BSC,"cocoa,blackberry,full bodied",3.75,3
1,Dominican Republic,0.76,BSC,"cocoa,vegetal,savory",3.50,3
2,Tanzania,0.76,BSC,"rich cocoa,fatty,bread",3.25,3
3,Peru,0.63,BSCL,"fruit,melon,roast",3.75,4
4,Bolivia,0.70,BSCL,"vegetal,nut",3.50,4


### 2. Create seperate CSV for all possible memorable characteristics.

Since the project requires understanding the individual characteristics rather than a list of characteristics, I decided to create a separate CSV containing every occurrence of memorable characteristics with is associated 'rating'.

In [32]:
#Create data frame for 'memorable_characteristics' to examine individual characteristics and rating
df_char = df_clean[['memorable_characteristics', 'rating']].copy()
df_char[['char1',  'char2', 'char3', 'char4']] = df_char['memorable_characteristics'].str.replace(', ', ',').str.split(',', expand=True)

#Create new data frame for individual characteristics
df_char_indv = pd.DataFrame(columns=['char1', 'rating'])

#Create smaller data frames to extract all individual characteristics
df_char1 = df_char[['char1', 'rating']].copy()
df_char2 = df_char[['char2', 'rating']].copy()
df_char2.rename(columns={'char2':'char1'}, inplace=True)
df_char3 = df_char[['char3', 'rating']].copy()
df_char3.rename(columns={'char3':'char1'}, inplace=True)
df_char4 = df_char[['char4', 'rating']].copy()
df_char4.rename(columns={'char4':'char1'}, inplace=True)

#Append created data frames to one data frame
df_char_indv = pd.DataFrame().append([df_char1, df_char2, df_char3, df_char4], ignore_index=True)
df_char_indv = df_char_indv.replace(to_replace='None', value=np.nan).dropna()
df_char_indv.rename(columns={'char1':'characteristic'}, inplace=True)


#Drop '' values from 'characteristic'
df_char_indv = df_char_indv.drop(df_char_indv[df_char_indv['characteristic'] == ''].index)
df_char_indv.loc[df_char_indv['characteristic'] == '']

#Reset Index
df_char_indv.index = range(len(df_char_indv.index))
df_char_indv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6398 entries, 0 to 6397
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   characteristic  6398 non-null   object 
 1   rating          6398 non-null   float64
dtypes: float64(1), object(1)
memory usage: 100.1+ KB


In [33]:
#Transform df_char_indv into CSV
df_char_indv.to_csv('individual_characteristics.csv')

# Final Cleaning
***

In [34]:
#Fix order of columns in dataframe
df_cleaned = df_clean[['bean_country', 'total_ingredients', 'ingredient_combination', 'cocoa_percent', 
                       'memorable_characteristics', 'rating']]
#Reset Index
df_cleaned.index = range(len(df_cleaned.index))

df_cleaned.head()

,bean_country,total_ingredients,ingredient_combination,cocoa_percent,memorable_characteristics,rating
0,Madagascar,3,BSC,0.76,"cocoa,blackberry,full bodied",3.75
1,Dominican Republic,3,BSC,0.76,"cocoa,vegetal,savory",3.50
2,Tanzania,3,BSC,0.76,"rich cocoa,fatty,bread",3.25
3,Peru,4,BSCL,0.63,"fruit,melon,roast",3.75
4,Bolivia,4,BSCL,0.70,"vegetal,nut",3.50


# Create CSV based on cleaned data frame.
***

In [35]:
#Transform data frame into CSV
df_clean.to_csv('cleaned_dark_chocolate_bar_ratings.csv')

### Sources
The Editors of Encyclopaedia Britannica. (2014, September 8). List of countries. Britannica. Retrieved March 19, 2021, from https://www.britannica.com/topic/list-of-countries-1993160